In [ ]:
# =====================================================
# IMPORTS
# =====================================================
import geopandas as gpd
import re
import pandas as pd
import folium
from folium.features import GeoJsonTooltip
import branca.colormap as cm
from shapely.ops import transform
from bs4 import BeautifulSoup
import re
import warnings
warnings.filterwarnings("ignore")


In [ ]:
def drop_z(geom):
    if geom is None:
        return None
    return transform(lambda x, y, z=None: (x, y), geom)


def html_a_texto(html):
    if html is None or str(html).strip() == "":
        return ""

    soup = BeautifulSoup(html, "html.parser")
    texto = soup.get_text(separator=" ")
    texto = texto.replace("\xa0", " ")
    texto = re.sub(r"\s+", " ", texto)

    return texto.strip()


def extraer_nombre_proyecto(texto, fallback=None):
    if texto is None or texto.strip() == "":
        return fallback

    patrones = [
        r"NOMBRE\s+DEL\s+PROYECTO\s*:\s*(.+)",
        r"NOMBRE\s+PROYECTO\s*:\s*(.+)"
    ]

    for pat in patrones:
        m = re.search(pat, texto, flags=re.IGNORECASE)
        if m:
            nombre = m.group(1)

            # Cortar cuando empieza otra sección típica
            nombre = re.split(
                r"PRIORIZADO|PLAN\s+PLURIANUAL|INVERSIÓN|ENTIDAD\s+CONTRATANTE",
                nombre,
                flags=re.IGNORECASE
            )[0]

            nombre = nombre.strip(" .:-")
            if nombre != "":
                return nombre.upper()

    return fallback


In [ ]:
atlantico = gpd.read_file("MAPAS/MGN_2/DPTO_CNMBR_ATLÁNTICO.shp")
vias      = gpd.read_file("MAPAS/VIAS/Linea.shp")
puntos    = gpd.read_file("MAPAS/PUNTOS/Puntos.shp")
nuevas    = gpd.read_file("MAPAS/NUEVAS/Vías Nuevas.shp")
nuevas = nuevas.rename(columns={'Nombre': 'name'})

In [ ]:
# Municipios
atlantico = atlantico.rename(columns={
    "MPIO_CCNCT": "COD_MUN",
    "MPIO_CNMBR": "NOM_MUN"
})
atlantico = atlantico[["COD_MUN", "NOM_MUN", "MPM", "geometry"]]

# Texto y categorías
for df in [vias, puntos]:
    df["name"] = df["name"].str.strip().str.upper()
    df["descripcion_txt"] = df["descriptio"].apply(html_a_texto).str.upper()
    df["Categoria"] = df["Categoria"].str.strip().str.upper()
    df["proyecto_nombre"] = df.apply(
        lambda r: extraer_nombre_proyecto(r["descripcion_txt"], r["name"]),
        axis=1
    )
    df = df.drop(columns=['name', 'altitude', 'alt_mode', 'descriptio', 'folders', 'time_begin', 'time_end', 'time_when', 'descripcion_txt'])
    print(df.head())
    print(df.columns)

nuevas['proyecto_nombre'] = nuevas['name'].str.strip().str.upper()
nuevas = nuevas.drop(columns=['id', 'name', 'Longitud (',])
print(nuevas.columns)

______